In [2]:
a=3
b=0.4
a*b

1.2000000000000002

In [4]:
str1="hello"
str2="How are you?"
sum_str=str1+", "+str2

print(sum_str)



hello, How are you?


In [5]:
str1="hello"
str2="How are you?"
sum_str=str1+",\n"+str2

print(sum_str)


hello,
How are you?


In [ ]:
a="HELLO"
print(a.lower())

b="HELLO"
print(b.lower())

c=len(a)
print(c)



In [3]:
num1=int(input("Enter your first number:"))
num2=int(input("Enter your second number:"))
sum_num=num1+num2

print (sum_num)


10


In [6]:
name=input("your name:")
str1="Hello "+name+"!"
str2=" Welcome to the planet earth."
sum_str=str1+str2

print(sum_str)


Hello Naoto!Welcome to the planet earth.


In [8]:
str1=float(input("Length:"))
str2=float(input("Width:"))

sum_str=2*(str1+str2)

print(sum_str)

16.0


In [9]:
import math
radius = float(input("Enter the radius of the circle: "))
circumference = 2 * math.pi * radius
area = math.pi * radius ** 2
print("Circumference of the circle is: ", round(circumference, 2))
print("Area of the circle is: ", round(area, 2))

Circumference of the circle is:  62.83
Area of the circle is:  314.16


In [10]:
import numpy as np
a=[1,2,3]
b=[4,5,6]
np.add(a,b)

array([5, 7, 9])

In [14]:
for x in range (0, 4, 2):
    print(x)

0
2


In [16]:
for x in range(0,19):
    print (x," squared is: ", x*x)

0  squared is:  0
1  squared is:  1
2  squared is:  4
3  squared is:  9
4  squared is:  16
5  squared is:  25
6  squared is:  36
7  squared is:  49
8  squared is:  64
9  squared is:  81
10  squared is:  100
11  squared is:  121
12  squared is:  144
13  squared is:  169
14  squared is:  196
15  squared is:  225
16  squared is:  256
17  squared is:  289
18  squared is:  324


In [17]:
a=1
while a<5:
    print(a)
    a=a+1

1
2
3
4


In [18]:
message = input("Enter the message: ")
num_times = int(input("Enter the number of times: "))
for i in range(num_times):
  print(message)


hello
hello
hello
hello
hello


In [19]:
x = 10
y = 15

if x > y:
  print("x is greater than y")
elif x < y:
  print("x is less than y")
else:
  print("x is equal to y")


x is less than y


In [20]:
x = int(input("Enter the first number: "))
y = int(input("Enter the second number: "))

if x > y:
    print("x is greater than y")
elif x < y:
    print("x is less than y")
else:
    print("x is equal to y")

x is equal to y


In [23]:
def calculate_bmi(weight, height):

  if weight <= 0 or height <= 0:
    print("Please provide a valid input.")
    return

  bmi = weight / (height * height)
  return round(bmi, 1)

def get_bmi_risk(bmi):

  if bmi < 18.5:
    return "Underweight"
  elif bmi < 25:
    return "Normal"
  elif bmi < 30:
    return "Overweight"
  else:
    return "Obese"

weight = float(input("Enter your weight in kilograms: "))
height = float(input("Enter your height in meters: "))

bmi = calculate_bmi(weight, height)

if bmi is not None:
  print(f"Your BMI is {bmi}.")
  print(f"Your BMI risk is {get_bmi_risk(bmi)}.")


Please provide a valid input.
